### Задача

**Вариант 2.** Проверить гипотезу о равенстве средних суммарных баллов за экзамены для разных этнических/национальных групп с помощью модели однофакторного дисперсионного анализа (ANOVA).

*   **Фактор (независимая переменная):** `race/ethnicity` (этническая/национальная группа).
*   **Выходная переменная (зависимая переменная):** Суммарный балл за три экзамена (`math score` + `reading score` + `writing score`).
*   **Указание:** Реализовать расчеты самостоятельно, а затем можно проверить результат с помощью библиотечной функции.

### Формулировка гипотез

*   **Нулевая гипотеза $ H_0 $:** Средние значения суммарных баллов одинаковы для всех этнических групп.
$ H_0: \mu_1 = \mu_2 = \mu_3 = ... = \mu_k $, где $k$ — количество групп.
*   **Альтернативная гипотеза $H_1$:** Хотя бы одно среднее значение суммарного балла отличается от других.

Уровень значимости $\alpha$ примем равным $0.05$.

### Шаг 1: Подготовка данных

In [10]:
import pandas as pd

df = pd.read_csv("exams_dataset.csv")

df["total_score"] = df["math score"] + df["reading score"] + df["writing score"]

print("Данные с новым столбцом 'total_score':")
print(
    df[
        [
            "race/ethnicity",
            "math score",
            "reading score",
            "writing score",
            "total_score",
        ]
    ].head()
)

Данные с новым столбцом 'total_score':
  race/ethnicity  math score  reading score  writing score  total_score
0        group B          67             91             84          242
1        group D          63             63             67          193
2        group A          73             83             85          241
3        group C          85             98             95          278
4        group B          75             57             63          195


### Шаг 2: Реализация дисперсионного анализа "вручную"

Дисперсионный анализ основан на разделении общей вариации данных на межгрупповую и внутригрупповую.

1.  **Общая сумма квадратов отклонений ($ SS_{total} $)**: Вариация всех данных относительно общего среднего.

$$ SS_{total} = \sum_{i=1}^{k} \sum_{j=1}^{n_i} (y_{ij} - \bar{y}_{..})^2 $$

2.  **Межгрупповая сумма квадратов ($ SS_{between} $)**: Вариация средних по группам относительно общего среднего.

$$ SS_{between} = \sum_{i=1}^{k} n_i (\bar{y}_{i.} - \bar{y}_{..})^2 $$

3.  **Внутригрупповая сумма квадратов ($ SS_{within} $)**: Сумма вариаций данных внутри каждой группы относительно среднего по группе.

$$ SS_{within} = \sum_{i=1}^{k} \sum_{j=1}^{n_i} (y_{ij} - \bar{y}_{i.})^2 $$

Где:
*   $ y_{ij} $ — $ j $-е наблюдение в $ i $-й группе.
*   $ \bar{y}_{..} $ — общее среднее по всем наблюдениям.
*   $ \bar{y}_{i.} $ — среднее по $ i $-й группе.
*   $ n_i $ — количество наблюдений в $ i $-й группе.
*   $ k $ — количество групп.

Затем мы вычисляем F-статистику:
$$ F = \frac{MS_{between}}{MS_{within}} = \frac{(N-k)}{(k-1)} \frac{SS_{between}}{SS_{within}} $$
Где $ N $ — общее число наблюдений, $ k-1 $ и $ N-k $ — степени свободы.

In [11]:
import numpy as np
from scipy import stats

groups = df.groupby("race/ethnicity")["total_score"]
group_data = [group for name, group in groups]

N = len(df)  # Общее количество наблюдений
k = len(group_data)  # Количество групп

grand_mean = df["total_score"].mean()  # Общее среднее
group_means = [group.mean() for group in group_data]  # Средние по группам
group_sizes = [len(group) for group in group_data]  # Размеры групп

ss_between = sum(
    group_sizes[i] * (group_means[i] - grand_mean) ** 2
    for i in range(k)
)

ss_within = sum(
    sum((obs - mean) ** 2 for obs in group)
    for group, mean in zip(group_data, group_means)
)

ss_total = sum((obs - grand_mean) ** 2 for obs in df["total_score"])

df_between = k - 1
df_within = N - k

ms_between = ss_between / df_between
ms_within = ss_within / df_within

f_statistic = ms_between / ms_within

p_value = stats.f.sf(
    f_statistic, df_between, df_within
)

print("--- РУЧНОЙ РАСЧЕТ ANOVA ---")
print(f"Общее количество наблюдений (N): {N}")
print(f"Количество групп (k): {k}\n")
print(f"SS_between (межгрупповая): {ss_between:.4f}")
print(f"SS_within (внутригрупповая): {ss_within:.4f}")
print(f"SS_total (общая): {ss_total:.4f}")
print(f"Проверка: SS_between + SS_within = {ss_between + ss_within:.4f}\n")
print(f"Степени свободы (межгрупповые): {df_between}")
print(f"Степени свободы (внутригрупповые): {df_within}\n")
print(f"MS_between: {ms_between:.4f}")
print(f"MS_within: {ms_within:.4f}\n")
print(f"F-статистика: {f_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

--- РУЧНОЙ РАСЧЕТ ANOVA ---
Общее количество наблюдений (N): 1000
Количество групп (k): 5

SS_between (межгрупповая): 112381.6433
SS_within (внутригрупповая): 1953413.1407
SS_total (общая): 2065794.7840
Проверка: SS_between + SS_within = 2065794.7840

Степени свободы (межгрупповые): 4
Степени свободы (внутригрупповые): 995

MS_between: 28095.4108
MS_within: 1963.2293

F-статистика: 14.3108
P-value: 0.0000


### Шаг 3: Проверка с помощью `scipy.stats`

Теперь проверим наши расчеты, используя готовую функцию `f_oneway` из библиотеки SciPy.

In [12]:
grouped_scores = [
    group["total_score"].values
    for name, group in df.groupby("race/ethnicity")
]

f_stat_lib, p_value_lib = stats.f_oneway(*grouped_scores)

print("--- ПРОВЕРКА С ПОМОЩЬЮ SCIPY ---")
print(f"F-статистика (из библиотеки): {f_stat_lib:.4f}")
print(f"P-value (из библиотеки): {p_value_lib:.4f}")

--- ПРОВЕРКА С ПОМОЩЬЮ SCIPY ---
F-статистика (из библиотеки): 14.3108
P-value (из библиотеки): 0.0000


### Шаг 4: Визуализация

In [13]:
import plotly.express as px

fig = px.box(
    df,
    x="race/ethnicity",
    y="total_score",
    color="race/ethnicity",
    category_orders={
        "race/ethnicity": sorted(df["race/ethnicity"].unique())
    },
    title="Распределение суммарных баллов по этническим группам",
    labels={
        "race/ethnicity": "Этническая/национальная группа",
        "total_score": "Суммарный балл",
    },
)
fig.update_layout(showlegend=False)
fig.show()

### Выводы

**Результаты статистического теста (ANOVA):**
*   **F-статистика:** $14.3108$
*   **P-value:** $0$ (или, если быть точнее, $p < 0.0001$)

Поскольку p-value значительно меньше уровня значимости, мы **отвергаем нулевую гипотезу ($H_0$)**.

**Заключение:**
Существуют статистически значимые доказательства того, что средний суммарный балл за экзамены **различается** хотя бы для одной из этнических/национальных групп. Другими словами, фактор `race/ethnicity` оказывает значимое влияние на суммарный балл студентов.